In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2
from glob import glob
import json
import sys
import os
import numpy as np
np.printoptions(decimals=1)

In [2]:
base='../data'
# Find all the json files recursively
json_files = glob(base + '/**/*.json', recursive=True)

print(f'Found {len(json_files)} json files')


Found 1672 json files


In [3]:
json_files = glob(base + '/**/*.json', recursive=True)

defectos_set=set()
for json_file in json_files:
    try:
        with open(json_file, 'r') as f:
            data = json.load(f)
            anotaciones_vistas=data["views_annotations"]
            for k,v in anotaciones_vistas.items():
                defectos=list(v.keys())
                defectos_set.update(defectos)
            
    except:
        print(f'Error reading {json_file}')
print(f'Found {len(defectos_set)} defectos')
print(defectos_set)

Error reading ../data\mandarins\clementine\manfredi2\light_scars\20250402_1321_000004.json
Error reading ../data\mandarins\clementine\manfredi2\light_scars\20250402_1323_002040.json
Error reading ../data\oranges\navel\manfredi2\damage_by_pliers\20250403_1053_002044.json
Error reading ../data\oranges\navel\manfredi2\deformed_counterpeduncle\20250403_0937_000027.json
Error reading ../data\oranges\navel\manfredi2\deformed_counterpeduncle\20250403_0938_000030.json
Error reading ../data\oranges\navel\manfredi2\light_scars\20250402_1345_000031.json
Error reading ../data\oranges\navel\manfredi2\light_scars\20250402_1347_002046.json
Error reading ../data\oranges\navel\manfredi2\light_scars\20250402_1348_002054.json
Error reading ../data\oranges\navel\manfredi2\light_scars\20250402_1349_002055.json
Error reading ../data\oranges\navel\manfredi2\mechanical_damage\20250402_1422_002082.json
Error reading ../data\oranges\navel\manfredi2\mechanical_damage\20250402_1423_002089.json
Error reading ../da

In [7]:
np.set_printoptions(precision=1)
json_files = glob(base + '/**/*.json', recursive=True)

defectos_set=list(defectos_set)
defectos_set.sort()
cuenta={ defecto:0 for defecto in defectos_set}

cuenta['sin_defecto']=0
cuenta['mas_de_una']=0

cuenta_vistas=0
anotaciones_dataset=[]
lista_sin_views=[]
listas_sospechosos={}
for defecto in defectos_set:
    listas_sospechosos[defecto]=[]
for json_file in json_files:

    with open(json_file, 'r') as f:
        data = json.load(f)
        if "views_annotations" not in data:
            print(json_file,"no views_annotations")
            lista_sin_views.append(json_file)
            continue
            
        anotaciones_vistas=data["views_annotations"]
        #print(anotaciones_vistas)
        anots_array=[]
        for nombre_vista,anotaciones_vista in anotaciones_vistas.items():
            #print(v)
            cuenta_vistas+=1
            
            nada=True
            c=0
            
            anots=[]
            m_defectos=[]
            for defecto,valor in anotaciones_vista.items():
                #print(valor)
                vint=int(valor)
                m_defectos.append(defecto)
                anots.append(vint)
                if vint>0:
                    cuenta[defecto]+=1
                    nada=False
                    c+=1
            if nada:
                cuenta['sin_defecto']+=1
            if c>1:
                cuenta['mas_de_una']+=1
            anots_array.append(anots)
        anots_array=np.array(anots_array)
        
        vistas_con_cada_defecto=np.mean(anots_array,axis=0)
        #print(vistas_con_cada_defecto)
        if np.any(vistas_con_cada_defecto>0.99):
            print(json_file)
            print(m_defectos)
            print(vistas_con_cada_defecto)
            
            def_idx=np.nonzero(vistas_con_cada_defecto>0.99)
            
            
            for idx in def_idx[0]:
                defecto=m_defectos[idx]
                listas_sospechosos[defecto].append(json_file)
                print(f'Adding {json_file} to {defecto}')
            print("=========================")    
        anotaciones_dataset.append(vistas_con_cada_defecto)

            
    # except:
    #     print(f'Error reading {json_file}')
print('Frutos sin vistas',len(lista_sin_views))

for k,v in listas_sospechosos.items():
    v.sort()
    
print("Total frutos: ", len(json_files))
print("Total Vistas: ", cuenta_vistas)
for d in cuenta:
    print(f'Vistas con {d}: {cuenta[d]}')


../data\mandarins\clementine\KDC\20220803_225931_CLEMENTINE_Encoder_SURFACE1\20250218_0846_000137.json
['bleaching', 'cracks', 'damage_by_pliers', 'deformed_counterpeduncle', 'deformed_peduncle', 'discarded', 'green', 'hollow', 'insect', 'light_scars', 'mechanical_damage', 'oil_spots', 'plu_stickers', 'reddish', 'rotten', 'scars', 'skin_breakdown', 'skin_creasing', 'surface']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Adding ../data\mandarins\clementine\KDC\20220803_225931_CLEMENTINE_Encoder_SURFACE1\20250218_0846_000137.json to surface
../data\mandarins\clementine\KDC\20220803_225931_CLEMENTINE_Encoder_SURFACE1\20250218_0846_000138.json
['bleaching', 'cracks', 'damage_by_pliers', 'deformed_counterpeduncle', 'deformed_peduncle', 'discarded', 'green', 'hollow', 'insect', 'light_scars', 'mechanical_damage', 'oil_spots', 'plu_stickers', 'reddish', 'rotten', 'scars', 'skin_breakdown', 'skin_creasing', 'surface']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Add

In [8]:
lista_sin_views

['../data\\mandarins\\clementine\\manfredi2\\light_scars\\20250402_1321_000004.json',
 '../data\\mandarins\\clementine\\manfredi2\\light_scars\\20250402_1323_002040.json',
 '../data\\oranges\\navel\\manfredi2\\damage_by_pliers\\20250403_1053_002044.json',
 '../data\\oranges\\navel\\manfredi2\\deformed_counterpeduncle\\20250403_0937_000027.json',
 '../data\\oranges\\navel\\manfredi2\\deformed_counterpeduncle\\20250403_0938_000030.json',
 '../data\\oranges\\navel\\manfredi2\\light_scars\\20250402_1345_000031.json',
 '../data\\oranges\\navel\\manfredi2\\light_scars\\20250402_1347_002046.json',
 '../data\\oranges\\navel\\manfredi2\\light_scars\\20250402_1348_002054.json',
 '../data\\oranges\\navel\\manfredi2\\light_scars\\20250402_1349_002055.json',
 '../data\\oranges\\navel\\manfredi2\\mechanical_damage\\20250402_1422_002082.json',
 '../data\\oranges\\navel\\manfredi2\\mechanical_damage\\20250402_1423_002089.json',
 '../data\\oranges\\navel\\manfredi2\\skin_creasing\\20250403_0954_000041.

In [9]:
print("Total frutos: ", len(json_files))
print("Total Vistas: ", cuenta_vistas)
for d in cuenta:
    print(f'Vistas con {d}: {cuenta[d]}')

Total frutos:  1672
Total Vistas:  16073
Vistas con bleaching: 258
Vistas con cracks: 153
Vistas con damage_by_pliers: 195
Vistas con deformed_counterpeduncle: 25
Vistas con deformed_peduncle: 281
Vistas con discarded: 0
Vistas con green: 733
Vistas con hollow: 365
Vistas con insect: 126
Vistas con light_scars: 1276
Vistas con mechanical_damage: 226
Vistas con oil_spots: 361
Vistas con plu_stickers: 446
Vistas con reddish: 222
Vistas con rotten: 658
Vistas con scars: 884
Vistas con skin_breakdown: 593
Vistas con skin_creasing: 99
Vistas con surface: 3143
Vistas con sin_defecto: 7325
Vistas con mas_de_una: 1246


In [41]:
anotaciones_dataset=np.array(anotaciones_dataset)
suma=np.sum(anotaciones_dataset==1,axis=0)


for defecto in defectos_set:
    print(f'{defecto}: {len(listas_sospechosos[defecto])} frutos sospechosas')
print("=======================================")
    

# print("Suma de defectos por vista")
# print(suma)


bleaching: 12 frutos sospechosas
cracks: 10 frutos sospechosas
damage_by_pliers: 2 frutos sospechosas
deformed_counterpeduncle: 0 frutos sospechosas
deformed_peduncle: 19 frutos sospechosas
discarded: 0 frutos sospechosas
green: 46 frutos sospechosas
hollow: 39 frutos sospechosas
insect: 7 frutos sospechosas
light_scars: 7 frutos sospechosas
mechanical_damage: 1 frutos sospechosas
oil_spots: 3 frutos sospechosas
plu_stickers: 1 frutos sospechosas
rotten: 26 frutos sospechosas
scars: 5 frutos sospechosas
skin_breakdown: 5 frutos sospechosas
skin_creasing: 9 frutos sospechosas
surface: 262 frutos sospechosas


In [39]:
listas_sospechosos['surface']

['../data/mandarins/clementine/KDC/20220803_225931_CLEMENTINE_Encoder_SURFACE1/20250218_0846_000137.json',
 '../data/mandarins/clementine/KDC/20220803_225931_CLEMENTINE_Encoder_SURFACE1/20250218_0846_000138.json',
 '../data/mandarins/clementine/KDC/20220803_225931_CLEMENTINE_Encoder_SURFACE1/20250218_0846_000139.json',
 '../data/mandarins/clementine/KDC/20220803_225931_CLEMENTINE_Encoder_SURFACE1/20250218_0846_000140.json',
 '../data/mandarins/clementine/KDC/20220803_225931_CLEMENTINE_Encoder_SURFACE1/20250218_0846_000141.json',
 '../data/mandarins/clementine/KDC/20220803_225931_CLEMENTINE_Encoder_SURFACE1/20250218_0846_000142.json',
 '../data/mandarins/clementine/KDC/20220803_225931_CLEMENTINE_Encoder_SURFACE1/20250218_0847_000143.json',
 '../data/mandarins/clementine/KDC/20220803_225931_CLEMENTINE_Encoder_SURFACE1/20250218_0847_000146.json',
 '../data/mandarins/clementine/KDC/20220803_225931_CLEMENTINE_Encoder_SURFACE1/20250218_0847_002104.json',
 '../data/mandarins/clementine/KDC/20

In [33]:
listas_sospechosos['oil_spots']

['../data/mandarins/clementine/KDC/20220804_221439_CLEMENTINE_ENCODER_Hollow/20250218_1113_000465.json',
 '../data/mandarins/nardocot/manfredi/20250220_1321_002258.json',
 '../data/oranges/navel/KDC/20220810_021928_Encoder_Buenas/20250218_1052_000420.json']

In [40]:
# Save lista s_sospechosos to json
with open('listas_sospechosos.json', 'w') as f:
    json.dump(listas_sospechosos, f, indent=4)